In [209]:
import numpy as np

## Linear Case

In [210]:
def deg_1_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_array = np.multiply.outer(a_range,u_range) # a x u
    bv_array = np.multiply.outer(b_range,v_range) # b x v
    
    temp_array = np.add.outer(au_array, bv_array) # a x u x b x v
    
    final_array = np.swapaxes(temp_array, 1,2)
    
    return final_array


def deg_1_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_1_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/(n*(n-1)))*np.sum(vec_A*vec_B,axis=2)
    
    return seq_array

In [274]:
deg_1_sqnc(-5,-5,4,4,21)

array([[[ 0.00000000e+00,  1.00000000e+00, -5.00000000e-01,
          2.50000000e-01, -1.25000000e-01,  6.25000000e-02,
         -3.12500000e-02,  1.56250000e-02, -7.81250000e-03,
          3.90625000e-03, -1.95312500e-03,  9.76562500e-04,
         -4.88281250e-04,  2.44140625e-04, -1.22070312e-04,
          6.10351562e-05, -3.05175781e-05,  1.52587891e-05,
         -7.62939453e-06,  3.81469727e-06, -1.90734863e-06]]])

## Quadratic Case

In [212]:
def deg_2_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_squared_matrix = np.multiply.outer(a_range, u_range*u_range) # a x u
    au_tensor = np.broadcast_to(au_squared_matrix,(seq_length,b_len,a_len,seq_length)) # k x b x a x k
    #k,b,a,k --> a,b,k,k
    
    bu_matrix = np.multiply.outer(b_range,u_range) # b_len x seq_length
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b_len x seq_len x seq_len
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,b_len,seq_length,seq_length))
    
    corrected_au_tensor = np.swapaxes(au_tensor, 0,2)
    #corrected_buv_tensor = np.swapaxes(buv_tensor, 2,3)
    
    tensor = corrected_au_tensor + buv_tensor
    
    P_array = np.swapaxes(tensor, 2,3)
    
    return P_array


def deg_2_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_2_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/((n*n)*(n-1)))*np.sum(vec_A*vec_B,axis=2)
    
    return seq_array

In [213]:
deg_2_sqnc(-24,-24,36,36,21)

array([[[ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12.,
         28., 14., 24., 24., 31., 18., 39., 20., 42.]]])

## Cubic Case

In [214]:
def deg_3_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_squared_matrix = np.multiply.outer(a_range, u_range*u_range*u_range) # a x u^3
    au_tensor = np.broadcast_to(au_squared_matrix,(seq_length,b_len,a_len,seq_length)) # k x b x a x k
    #k,b,a,k --> a,b,k,k
    
    bu_matrix = np.multiply.outer(b_range,u_range*u_range) # b x u^2
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b x u^2 x v
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,b_len,seq_length,seq_length)) # a x b x k x k
    
    corrected_au_tensor = np.swapaxes(au_tensor, 0,2)
    #corrected_buv_tensor = np.swapaxes(buv_tensor, 2,3)
    
    tensor = corrected_au_tensor + buv_tensor
    
    P_array = np.swapaxes(tensor, 2,3)
    
    return P_array


def deg_3_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_3_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/((n**3)*(n-1)))*np.sum(vec_A*vec_B,axis=2)
    
    return seq_array

In [215]:
deg_3_sqnc(6,6,18,18,21)

array([[[0.00000000e+00, 1.00000000e+00, 3.00000000e+00, 9.00000000e+00,
         2.70000000e+01, 8.10000000e+01, 2.43000000e+02, 7.29000000e+02,
         2.18700000e+03, 6.56100000e+03, 1.96830000e+04, 5.90490000e+04,
         1.77147000e+05, 5.31441000e+05, 1.59432300e+06, 4.78296900e+06,
         1.43489070e+07, 4.30467210e+07, 1.29140163e+08, 3.87420489e+08,
         1.16226147e+09]]])

## Catalan Numbers

In [216]:
def catalan(length):
    catlist = np.zeros(length)
    catlist[1] = 1
    for n in range(2,length):
        vec_A = catlist[1:n]
        vec_B = catlist[n-1:0:-1]
        catlist[n] = np.dot(vec_A,vec_B)

    return catlist

In [217]:
catalan(21)[1]

1.0

## Checking for Geometric Sequences

Just some preliminary goofing off

In [218]:
A = np.arange(1,13).reshape(2,2,3)

In [219]:
A[:,:,1:]/A[:,:,:2]

array([[[2.        , 1.5       ],
        [1.25      , 1.2       ]],

       [[1.14285714, 1.125     ],
        [1.1       , 1.09090909]]])

In [221]:
A.shape

(2, 2, 3)

In [222]:
deg_2_sqnc(-20,20,-20,20,21).shape[2]

21

In [245]:
def array_div(A): # input is 3d array
    orig_sqnc_len = A.shape[2]
    
    shift_1 = A[:,:,2::] # starts each sequence at their first index
    shift_2 = A[:,:,1:orig_sqnc_len-1:] # then at their second index
    
    div_array = np.divide(shift_1,shift_2, out=np.zeros_like(shift_1), where=shift_2!=0)
    # puts a zero wherever it had to divide by zero
    
    return div_array

In [249]:
def geom_sqnc_check(A):
    orig_array = array_div(A)
    ax_1 = orig_array.shape[0]
    ax_2 = orig_array.shape[1]
    
    bool_array = np.full((ax_1,ax_2), False)
    
    for a in range(ax_1): # no idea how to vectorize this
        for b in range(ax_2):
            sqnc = orig_array[a,b]
            elt_1 = sqnc[0]
            if np.count_nonzero(sqnc == elt_1) > 2: # found some cases where the first number appears exactly twice
                bool_array[a,b] = True
            else:
                continue
    arrays = np.where(bool_array) # two arrays of coordinates where bool_array is True
    x_coords = arrays[0]
    y_coords = arrays[1]
    length = len(x_coords) # they're both the same length, so I just picked one
    coord_list = [(x_coords[n],y_coords[n]) for n in range(length)]
    
    return coord_list

In [250]:
geom_sqnc_check(deg_3_sqnc(-20,20,-60,60,21))

[(0, 0),
 (0, 80),
 (1, 3),
 (1, 79),
 (2, 6),
 (2, 78),
 (3, 9),
 (3, 77),
 (4, 12),
 (4, 76),
 (5, 15),
 (5, 75),
 (6, 18),
 (6, 74),
 (7, 21),
 (7, 73),
 (8, 24),
 (8, 72),
 (9, 27),
 (9, 71),
 (10, 30),
 (10, 70),
 (11, 33),
 (11, 69),
 (12, 36),
 (12, 68),
 (13, 39),
 (13, 67),
 (14, 42),
 (14, 66),
 (15, 45),
 (15, 65),
 (16, 48),
 (16, 64),
 (17, 51),
 (17, 63),
 (18, 54),
 (18, 62),
 (19, 57),
 (19, 61),
 (20, 60),
 (21, 59),
 (21, 63),
 (22, 58),
 (22, 66),
 (23, 57),
 (23, 69),
 (24, 56),
 (24, 72),
 (25, 55),
 (25, 75),
 (26, 54),
 (26, 78),
 (27, 53),
 (27, 81),
 (28, 52),
 (28, 84),
 (29, 51),
 (29, 87),
 (30, 50),
 (30, 90),
 (31, 49),
 (31, 93),
 (32, 48),
 (32, 96),
 (33, 47),
 (33, 99),
 (34, 46),
 (34, 102),
 (35, 45),
 (35, 105),
 (36, 44),
 (36, 108),
 (37, 43),
 (37, 111),
 (38, 42),
 (38, 114),
 (39, 41),
 (39, 117),
 (40, 40),
 (40, 120)]

The cubic function appears to produce geometric sequences forever ordered pair $(a,b) = (a,3a)$ where $a \in \mathbb{Z}$.

In [277]:
geom_sqnc_check(deg_1_sqnc(-10,10,-10,10,21))[:30]

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8)]

It looks like the linear case *only* produces geometric sequences or sequences of zeros!

In [271]:
deg_1_sqnc(-10,10,-10,10,21)[13,18]

array([0.00000000e+00, 1.00000000e+00, 5.50000000e+00, 3.02500000e+01,
       1.66375000e+02, 9.15062500e+02, 5.03284375e+03, 2.76806406e+04,
       1.52243523e+05, 8.37339379e+05, 4.60536658e+06, 2.53295162e+07,
       1.39312339e+08, 7.66217865e+08, 4.21419826e+09, 2.31780904e+10,
       1.27479497e+11, 7.01137235e+11, 3.85625480e+12, 2.12094014e+13,
       1.16651708e+14])

In [272]:
array_div(deg_1_sqnc(-10,10,-10,10,21))[13,18]

array([5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5,
       5.5, 5.5, 5.5, 5.5, 5.5, 5.5])

In [257]:
deg_3_sqnc(-20,20,-60,60,21)[40,120]

array([0.e+00, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06,
       1.e+07, 1.e+08, 1.e+09, 1.e+10, 1.e+11, 1.e+12, 1.e+13, 1.e+14,
       1.e+15, 1.e+16, 1.e+17, 1.e+18, 1.e+19])

In [255]:
array_div(deg_3_sqnc(-20,20,-60,60,21))[0,0]

array([-10., -10., -10., -10., -10., -10., -10., -10., -10., -10., -10.,
       -10., -10., -10., -10., -10., -10., -10., -10.])

In [162]:
%timeit geom_sqnc_check(deg_3_sqnc(-20,20,-20,20,21))

8.18 ms ± 25.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [278]:
geom_sqnc_check(deg_2_sqnc(-10,10,-10,10,21))

[(0, 0),
 (0, 20),
 (1, 1),
 (1, 19),
 (2, 2),
 (2, 18),
 (3, 3),
 (3, 17),
 (4, 4),
 (4, 16),
 (5, 5),
 (5, 15),
 (6, 6),
 (6, 14),
 (7, 7),
 (7, 13),
 (8, 8),
 (8, 12),
 (9, 9),
 (9, 11),
 (10, 10),
 (11, 9),
 (11, 11),
 (12, 8),
 (12, 12),
 (13, 7),
 (13, 13),
 (14, 6),
 (14, 14),
 (15, 5),
 (15, 15),
 (16, 4),
 (16, 16),
 (17, 3),
 (17, 17),
 (18, 2),
 (18, 18),
 (19, 1),
 (19, 19),
 (20, 0),
 (20, 20)]

In [282]:
array_div(deg_2_sqnc(-10,10,-10,10,21))[14,14]

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2.])

In [283]:
array_div(deg_2_sqnc(-10,10,-10,10,21))[14,6]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

Looks like the quadratic function produces a geometric sequence for each ordered pair $(a,b) = (a,a)$ and sequences of zeros for $(a,b) = (a,-a)$., $a 

#### Some Ideas

After finding a function that finds the geometric sequences, try dividing everything by sequences of catalan numbers or sequences of sums of factors, then put it through the function. This way we can pinpoint the combo-sequences.